In [ ]:
import numpy as np
import pandas as pd
import datetime

In [ ]:
def detect_status(equip_status,time):
    list_id=[0]*20
    list_status=[0]*20
    list_level=[0]*20
    list_time=[0]*20
    list_process=[0]*20
    low,med=0,0
    for i in range(equip_status.shape[0]):
        if equip_status.iloc[i][2]==time and equip_status.iloc[i][1]!='Normal':
            if equip_status.iloc[i][5]=='Low':
                low=low+1
                j=0
            elif equip_status.iloc[i][5]=='Medium':
                med=med+1
                j=low
            elif equip_status.iloc[i][5]=='High':
                j=low+med
            list_id.insert(j,equip_status.iloc[i][0])
            list_status.insert(j,equip_status.iloc[i][1])
            list_level.insert(j,equip_status.iloc[i][7])
            list_time.insert(j, equip_status.iloc[i][2])
            list_process.insert(j, equip_status.iloc[i][6])
    equip_data=pd.DataFrame({'EquipmentID':list_id,'EquipmentState':list_status,'Level':list_level,'Time':list_time,'Process':list_process})
    return equip_data

#和equip_info做join得到zone_num
def equip_info_join(equip_data,equip_info):
    return pd.merge(left=equip_data, right=equip_info, how='inner', on=['EquipmentID','Process'])


def action_join(equip_all,action,identity):
    dataframe=pd.merge(left=equip_all,right=action,how='inner',on=['EquipmentState','Process'])
    return dataframe

def identity_join(all,entity):
    dataframe=pd.DataFrame(all,columns=['EmployeeRole'])
    employee_final=pd.merge(left=dataframe,right=entity,how='inner',on='EmployeeRole')
    return pd.DataFrame(employee_final,columns=['Email','ClientMacAddr'])

def find_employee(dataframe,time):
    list_1=[]
    list_2=[]
    list_3=[]
    list_4=[]
    time=datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    for index,row in dataframe.iterrows():
        if (time-datetime.datetime.strptime(row['date']+' '+row['time'],'%Y-%m-%d %H:%M:%S')).seconds<=600:
            list_1.append(row['ClientMacAddr'])
            list_2.append(row['lat'])
            list_3.append(row['lng'])
            list_4.append(row['Level'])
    data=pd.DataFrame({'ClientMacAddr':list_1,'lat':list_2,'lng':list_3,'Floor':list_4})
    return data

def original_join(original,final):
    return pd.merge(left=original,right=final,how='inner',on='ClientMacAddr')

def assign_zones(dataframe, x_min, x_max, y_min, y_max, k):
    # Define the x and y boundaries of the grid
    x_step = (x_max-x_min)/k
    y_step = (y_max-y_min)/k

    # assign zone number for the given dataframe
    zone_num = 1
    for i in range(k):
        for j in range(k):
            x = x_min + i * x_step
            y = y_min + j * y_step
            mask = ((dataframe['lng'] >= x) & (dataframe['lng'] <= x + x_step) &
                    (dataframe['lat'] >= y) & (dataframe['lat'] <= y + y_step))
            dataframe.loc[mask, 'zone'] = zone_num
            zone_num += 1
    dataframe['zone'] = dataframe['zone'].astype(int)
    return dataframe




In [ ]:
def initial(k):
    frame_1=np.zeros([k,k])
    frame_2=np.zeros([k,k])
    frame_3=np.zeros([k,k])
    frame_4=np.zeros([k,k])
    list=[]
    list.append(frame_1)
    list.append(frame_2)
    list.append(frame_3)
    list.append(frame_4)
    return list

poly=[[[0,2],[0,8],[12,7],[14,19],[19,19],[17,0],[0,2]]]

def find_email(zone,security_final):
    for index,row in security_final.iterrows():
        if row['zone']==zone:
            return row['ClientMacAddr']
#将zone_number转换为坐标
def transfer_1(num,k):
    yy=num%k-1
    if yy<0:
        yy=19
    xx=int((num-yy)/k)
    return(xx,yy)
def transfer(num,k):
    yy=num%k-1
    if yy<0:
        yy=19
    xx=int(num/k)
    return(xx,yy)

def de_transfer(xx,yy,k):
    return xx*k+yy+1

#将楼层转换为数字
def tran_floor(string):
    if string=='Ground Floor':
        return 0
    elif string=='1st Floor':
        return 1
    elif string=='2nd Floor':
        return 2
    elif string=='3rd Floor':
        return 3

#楼梯位置初始化
stair=pd.read_csv("D:/USC/stair.csv")
list_stair=[]
for i in range(0,3):
    point=(transfer_1(stair.iloc[i][1],20)[0],transfer_1(stair.iloc[i][1],20)[1])
    list_stair.append(point)


def floor_info(dataframe,k,list):
    for i in range(0,dataframe.shape[0]):
        point=transfer(dataframe.iloc[i][5],k)
        list[tran_floor(dataframe.iloc[i][3])][point[0]][point[1]]=1
    return list


def isRayIntersectsSegment(xx,yy,s_poi,e_poi): #[x,y] [lng,lat]
    #输入：判断点，边起点，边终点，都是[lng,lat]格式数组
    if s_poi[1]==e_poi[1]: #排除与射线平行、重合，线段首尾端点重合的情况
        return False
    if s_poi[1]>yy and e_poi[1]>yy: #线段在射线上边
        return False
    if s_poi[1]<yy and e_poi[1]<yy: #线段在射线下边
        return False
    #if s_poi[1]==yy and e_poi[1]>yy: #交点为下端点，对应spoint
        #return False
    if e_poi[1]==yy and s_poi[1]>yy: #交点为下端点，对应epoint
        return False
    if s_poi[0]<xx and e_poi[1]<xx: #线段在射线左边
        return False

    xseg=e_poi[0]-(e_poi[0]-s_poi[0])*(e_poi[1]-yy)/(e_poi[1]-s_poi[1]) #求交
    if xseg<xx: #交点在射线起点的左侧
        return False
    return True  #排除上述情况之后

def isPoiWithinPoly(xx,yy,poly):
    #输入：点，多边形三维数组
    #poly=[[[x1,y1],[x2,y2],……,[xn,yn],[x1,y1]],[[w1,t1],……[wk,tk]]] 三维数组

    #可以先判断点是否在外包矩形内
    #if not isPoiWithinBox(poi,mbr=[[0,0],[180,90]]): return False
    #但算最小外包矩形本身需要循环边，会造成开销，本处略去
    sinsc=0 #交点个数
    for epoly in poly: #循环每条边的曲线->each polygon 是二维数组[[x1,y1],…[xn,yn]]
        for i in range(len(epoly)-1): #[0,len-1]
            s_poi=epoly[i]
            e_poi=epoly[i+1]
            if isRayIntersectsSegment(xx,yy,s_poi,e_poi):
                sinsc+=1 #有交点就加1

    return True if sinsc%2==1 else  False

#以点(xx,yy)为中心搜索
def search_floor(k,floor,xx,yy):
    for i in range(xx-k,xx+k+1):
        for j in range(yy-k,yy+k+1):
            if isPoiWithinPoly(i,j,poly)==True:
                #print(i,j)
                if list[floor-1][i][j]==1:
                    return (i,j)
    return 0

#print(search_floor(3,2,1,10))
#k:搜索范围  m:上下m层楼检索
def search_total(k,floor,m,sub,xx,yy,stairx,stairy):
    #本层楼再扩大范围检索
    id=search_floor(k,floor,xx,yy)
    if id!=0:
        return id
    #上下一层楼检索
    if m==1:
        #print("上下1层")
        #print(k-sub)
        if floor + 1 < 4:
            id = search_floor(k - sub, floor + 1,stairx,stairy)
            if id != 0:
                return id
        if floor - 1 >=0:
            id = search_floor(k - sub, floor - 1,stairx,stairy)
            if id != 0:
                return id
    if m==2:
        #print("上下2层")
        if floor + 1 < 4:
            id = search_floor(k - sub, floor + 1,stairx,stairy)
            if id != 0:
                return id
        if floor - 1 >= 0:
            id = search_floor(k - sub, floor - 1,stairx,stairy)
            if id != 0:
                return id
        if floor + 2 < 4:
            id = search_floor(k - sub-1, floor + 2,stairx,stairy)
            if id != 0:
                return id
        if floor - 2 >= 0:
            id = search_floor(k - sub-1, floor - 2,stairx,stairy)
            if id != 0:
                return id
    if m==3:
        #print("上下3层")
        if floor + 1 < 4:
            id = search_floor(k - sub, floor + 1,stairx,stairy)
            if id != 0:
                return id
        if floor - 1 >= 0:
            id = search_floor(k - sub, floor - 1,stairx,stairy)
            if id != 0:
                return id
        if floor + 2 < 4:
            id = search_floor(k - sub-1, floor + 2,stairx,stairy)
            if id != 0:
                return id
        if floor - 2 >= 0:
            id = search_floor(k - sub-1, floor - 2,stairx,stairy)
            if id != 0:
                return id
        if floor + 3 < 4:
            id = search_floor(k - sub-2, floor + 3,stairx,stairy)
            if id != 0:
                return id
        if floor - 3 >= 0:
            id = search_floor(k - sub-2, floor - 3,stairx,stairy)
            if id != 0:
                return id
    return 0


def search(xx,yy,floor):
    #首先搜索本层楼
    #计算k:初始值为该格子到最近的电梯的距离
    if xx<=10 and yy<=10:
        sub=max(xx,10-yy)
        stairx, stairy = list_stair[0][0],list_stair[0][1]
    if xx>10 and yy<=10:
        sub=max(17-xx,yy)
        stairx, stairy = list_stair[1][0],list_stair[1][1]
    if xx>10 and yy>10:
        sub=max(19-xx,19-yy)
        stairx, stairy = list_stair[2][0],list_stair[2][1]
    k=sub
    for i in range(1,k+1):
        id=search_floor(i,floor,xx,yy)
        if id!=0:
            return id
    k=k+1
    m=1
    #若本层楼检索到最近的楼梯处依旧无人，则开始检索上下m层楼是否有人
    while k<=18:
        id=search_total(k,floor,m,sub,xx,yy,stairx,stairy)
        if id==0:
            k=k+1
            m=m+1
            if m>3:
                m=3
        else:
            return id
    return 0



In [ ]:
original=pd.read_csv("D:/USC/2019-08-13-cleaned.csv")
entity=pd.read_csv("D:/USC/employee_identity-2.csv")
action=pd.read_csv("D:/USC/employee_action.csv")
equip_status=pd.read_csv("D:/USC/equip_status.csv")
equip_info=pd.read_csv("D:/USC/equip_info.csv")


In [ ]:
#data = find_employee(original, '2019-08-13 10:58:07')

In [ ]:
list=initial(20)

In [ ]:
def total(equip_status,equip_info,entity,action,original,time):
    list_EquipID=[]
    list_ClientMacAddr=[]
    i=0
    data = find_employee(original, time)
    status_data=detect_status(equip_status,time)
    while(status_data.iloc[i][0]):
        equip_data=pd.DataFrame({'EquipmentID':[status_data.iloc[i][0]],'EquipmentState':[status_data.iloc[i][1]],'Level':[status_data.iloc[i][2]],'Time':[status_data.iloc[i][3]],'Process':[status_data.iloc[i][4]]})
        equip_all = equip_info_join(equip_data, equip_info)
        all = action_join(equip_all, action, entity)
        dataframe_final = identity_join(all, entity)

        data_person = original_join(data, dataframe_final)
        dataframe = assign_zones(data_person, -0.9332446313023279, -0.9321732468412318, 51.460257062787136,
                                 51.46094763183397, 20)

        floor_info(dataframe,20,list)

        point = search(transfer(equip_all.iloc[0][6], 20)[0], transfer(equip_all.iloc[0][6], 20)[1], tran_floor(equip_all.iloc[0][2]))
        print(point)
        e = find_email(de_transfer(point[0], point[1], 20), dataframe)
        print(e)
        list_EquipID.append(equip_data.iloc[0][0])
        list_ClientMacAddr.append(e)
        i=i+1
    return pd.DataFrame({'EquipmentID':list_EquipID,'ClientMacAddr':list_ClientMacAddr}).iloc[::-1].reset_index().drop('index',axis=1)

In [ ]:
macid=total(equip_status,equip_info,entity,action,original,'2019-08-13 10:58:07')

(16, 16)
80:58:f8:38:3c:47
(17, 18)
80:58:f8:5a:0d:88
(4, 3)
88:b4:a6:49:88:11


In [ ]:
macid

,EquipmentID,ClientMacAddr
0,P001,88:b4:a6:49:88:11
1,P002,80:58:f8:5a:0d:88
2,P004,80:58:f8:38:3c:47


In [ ]:
def pre_work_email(equip_status,equip_info,entity,action,macid):
    equip_status['Priority'] = equip_status['Prority']
    equip_status = equip_status.drop('Prority', axis=1)
    equip_status = equip_status[~equip_status['Priority'].isna()]
    equip = pd.merge(equip_status, equip_info, on=['EquipmentID','Process'], how='left')
    equip = equip[['EquipmentID', 'EquipmentState', 'localtime', 'Priority', 'Process', 'Level', 'ZoneID']]
    employee = pd.merge(macid, entity, on='ClientMacAddr', how='left')
    equip_employee = pd.merge(equip, employee, on='EquipmentID', how='left')
    equip_employee_action = pd.merge(equip_employee, action, on=['EmployeeRole', 'Process', 'EquipmentState', 'Priority'], how='left')
    return equip_employee_action

In [ ]:
def employee_email_content (equip_employee_action):
    for i, row in equip_employee_action.iterrows():
        print('Send to:' + row['Email'])
        employee = f"Dear {row['EmployeeRole']} {row['EmployeeName']}, \n"
        print(employee)
        alert = f"The state of equipment ({row['EquipmentID']}) changes to {row['EquipmentState']} at {row['localtime']}."
        print(alert)
        location = f"The location is {row['Level']}, and the zone id is {row['ZoneID']}."
        print(location)
        action = f"What you need to do is the following step(s): \n{row['Action']} \n"
        print(action)
        print('Please investigate on-site as soon as possible! \n\n')

In [ ]:
employee_email_content(pre_work_email(equip_status,equip_info,entity,action,macid))

Send to:lorenzo.garcia@company.com
Dear Scientist Lorenzo Garcia, 

The state of equipment (P001) changes to Hazardous Leak at 2019-08-13 10:58:07.
The location is 3rd Floor, and the zone id is 107.
What you need to do is the following step(s): 
• Ring alarm
• Email / Text
• Shutdown 

Please investigate on-site as soon as possible! 


Send to:sofia.smith@company.com
Dear Bioproduction Technologist Sofia Smith, 

The state of equipment (P002) changes to Calibration Needed at 2019-08-13 10:58:07.
The location is 3rd Floor, and the zone id is 375.
What you need to do is the following step(s): 
Notice of schedule 

Please investigate on-site as soon as possible! 


Send to:giulia.garcia@company.com
Dear Production Operator Giulia Garcia, 

The state of equipment (P004) changes to Regular Maintenance Needed at 2019-08-13 10:58:07.
The location is 3rd Floor, and the zone id is 360.
What you need to do is the following step(s): 
Refresh Maintenance Log  

Please investigate on-site as soon a